In [ ]:
# 데이터베이스에 대한 자연어 질의를 SQL 질의로 변경해주는 소규모 LLM 모델 개발  # 참고: LLM을 활용한 실전 AI 애플리케이션 개발
# sLLM이 생성한 SQL이 데이터 요청을 잘 해결하는지 GPT-4 API를 활용해 성능을 평가
# 데이터 출처: https://huggingface.co/datasets/shangrilar/ko_text2sql

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 huggingface_hub==0.22.2 autotrain-advanced==0.7.77 -qqq
!pip install --upgrade huggingface-hub -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 92.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.5 MB/s eta 0:0

In [ ]:
# 미세조정 수행 전 기초 모델 불러와서 평가하기
## 기초 모델 생성
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

def make_inference_pipeline(model_id):  # 입력한 모델 id에 맞춰 토크나이저와 모델을 불러오고 하나의 파이프라인을 만들어서 반환하는 함수
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return pipe

In [ ]:
model_id = 'beomi/Yi-Ko-6B'  # Beomi님의 한/영 혼합 데이터셋으로 사전 학습한 모델
hf_pipe = make_inference_pipeline(model_id)  # 'beomi/Yi-Ko-6B'모델로 파이프라인을 만들고 hf_pipe 변수에 저장

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/2.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/643M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
example = """

당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
    player_id INT PRIMARY KEY AUTO_INCREMENT,
    username VARCHAR(255) UNIQUE NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    date_joined DATETIME NOT NULL,
    last_login DATETIME
);

### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL:

"""

hf_pipe(example, do_sample=False, return_full_text=False, max_length=512, truncation=True)  # 예시 데이터를 작성해서 hf_pipe에 입력하고 결과 확인

# 쿼리를 잘 생성하지만 이후에 반복으로 '봇의 결과: ~~'가 계속 생성되어 형식에 맞춰 답변하도록 추가적인 학습이 필요

[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n\nSELECT 

In [ ]:
# 필요한 함수 생성
## 1. SQL 프롬프트 생성 함수
def make_prompt(ddl, question, query=''):  # DDL, 질문(question), 쿼리를 입력으로 받아 SQL 생성 프롬프트를 만듦
    prompt = f"""
                당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

                ### DDL:
                {ddl}

                ### Question:
                {question}

                ### SQL:
                {query}
            """  # 프롬프트를 구성:
                 # - SQL 생성에 필요한 정보를 포함하며, DDL(테이블 정의), 문제(질문), 이미 제공된 SQL(query, 선택사항)을 포함
                 # - 쿼리(query)는 기본값으로 빈 문자열('')이 사용됨
    return prompt  # 최종적으로 구성된 프롬프트를 반환


## 2. 평가를 위한 요청 jsonl 작성 함수
import json
import pandas as pd
from pathlib import Path

def make_requests_for_gpt_evaluation(df, filename, dir='requests'):  # DataFrame(df), 저장할 파일 이름(filename), 저장 경로(dir)를 입력으로 받음
    if not Path(dir).exists():
        Path(dir).mkdir(parents=True)  # 만약 디렉토리가 없으면 생성
    prompts = []  # 빈 프롬프트 리스트 생성
    for idx, row in df.iterrows():  # DataFrame(df)의 각 행(row)에 대해 반복
        prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""

                        DDL: {row['context']}
                        Question: {row['question']}
                        gt_sql: {row['answer']}
                        gen_sql: {row['gen_sql']}"""

                        )

                        # 각 행의 정보를 사용해 평가 프롬프트를 생성
                        # - row['context']: DDL 정의
                        # - row['question']: 질문
                        # - row['answer']: 정답 SQL(gt_sql)
                        # - row['gen_sql']: 생성된 SQL(gen_sql)
                        # 프롬프트에 평가 조건을 명시적으로 포함

    jobs = [{"model": "gpt-4-turbo-preview", "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
    with open(Path(dir, filename), "w") as f:  # 지정된 디렉토리(dir)와 파일 이름(filename)으로 파일을 열기(쓰기 모드)
        for job in jobs:  # 생성된 요청(job)을 반복 처리
             json_string = json.dumps(job)  # 요청(job)을 JSON 문자열로 변환
             f.write(json_string + "\n")    # JSON 문자열을 파일에 저장하며, 각 요청은 줄바꿈으로 구분

    # 각 프롬프트를 기반으로 GPT 요청(job) 생성
    # - 모델: gpt-4-turbo-preview
    # - 메시지 형식: system 역할 메시지에 프롬프트 추가
    # - 응답 형식(response_format): JSON 객체 형태로 지정


## 3. 결과 jsonl 파일을 csv로 변환하는 함수
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    # JSONL 입력 파일(input_file), 변환된 CSV 출력 파일(output_file), 그리고 컬럼 이름(prompt_column, response_column)을 입력으로 받음
    prompts = []    # 빈 프롬프트 리스트 생성
    responses = []  # 빈 응답 리스트 생성
    with open(input_file, 'r') as json_file:  # JSONL 파일을 읽기 모드로 열기
        for data in json_file:  # JSONL 파일의 각 줄에 대해 반복 처리
            prompts.append(json.loads(data)[0]['messages'][0]['content'])  # 각 JSONL 데이터의 첫 번째 메시지에서 프롬프트(content)를 추출해 리스트에 추가
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])  # JSONL 데이터의 두 번째 메시지에서 응답(content)을 추출해 리스트에 추가

    df = pd.DataFrame({prompt_column: prompts, response_column: responses})   # 추출된 프롬프트와 응답 데이터를 DataFrame으로 변환
                                                                              # - prompt_column 이름으로 프롬프트 저장
                                                                              # - response_column 이름으로 응답 저장
    df.to_csv(output_file, index=False)  # 변환된 DataFrame을 지정된 CSV 파일(output_file)로 저장
    return df  # 최종적으로 변환된 DataFrame을 반환

In [ ]:
!pip install --upgrade datasets huggingface_hub -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.7.77 requires datasets[vision]~=2.19.0, but you have datasets 3.2.0 which is incompatible.
autotrain-advanced 0.7.77 requires huggingface-hub==0.22.2, but you have huggingface-hub 0.27.1 which is incompatible.
autotrain-advanced 0.7.77 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
autotrain-advanced 0.7.77 requires tqdm==4.66.2, but you have tqdm 4.67.1 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.3.1 wh

In [ ]:
!mkdir results

In [ ]:
from datasets import load_dataset
# 데이터셋 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test']  # shangrilar/ko_text2sql 데이터셋의 origin 버전에서 test 스플릿 데이터를 불러옴
df = df.to_pandas()  # 허깅페이스 데이터셋 형식을 판다스 데이터프레임 형식으로 변환
for idx, row in df.iterrows():  # 판다스 데이터프레임의 각 행을 반복(iterate)하면서 인덱스와 데이터를 가져옴
    prompt = make_prompt(row['context'], row['question'])  # 각 행의 context(DDL 정보)와 question(SQL 질문)을 기반으로 프롬프트를 생성
    df.loc[idx, 'prompt'] = prompt  # 생성된 프롬프트를 prompt 컬럼에 저장함  # .loc를 사용하여 특정 인덱스(idx)에 값을 지정

# sql 생성
gen_sqls = hf_pipe(df['prompt'].tolist(),   # pipeline(hf_pipe)을 사용하여 프롬프트로부터 SQL 쿼리를 생성
                   do_sample=False,         # 확률적 샘플링 비활성화
                   return_full_text=False,  # 입력 텍스트를 제외하고 모델 출력만 반환
                   max_length=512,          # 출력 텍스트 길이 512 토큰으로 제한
                   truncation=True)         # 입력 텍스트가 너무 길 경우 최대 길이 기준으로 잘라내서 모델에 전달

gen_sqls = [x[0]['generated_text'] for x in gen_sqls]  # 생성된 결과에서 SQL 텍스트만 추출
df['gen_sql'] = gen_sqls

# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)  # 생성된 SQL과 실제 정답(SQL)을 평가하기 위해 JSONL 형식의 요청 파일을 생성

README.md:   0%|          | 0.00/281 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/25.6M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/38246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

In [ ]:
# OpenAI API키 입력
import os
os.environ["OPENAI_API_KEY"] = "my_key"

In [ ]:
# GPT-4 평가 수행
!python api_request_parallel_processor.py \                 # py파일 드라이브에 업로드 후 실행
--requests_filepath requests/{eval_filepath}  \             # 평가를 위해 요청할 JSONL 파일의 경로를 지정  # 이전 코드에서 생성한 eval_filepath 변수를 사용해 경로를 지정
--save_filepath results/{eval_filepath} \                   # OpenAI API 응답 결과를 저장할 파일 경로를 지정
--request_url https://api.openai.com/v1/chat/completions \  # OpenAI API의 엔드포인트 URL
--max_requests_per_minute 2500 \                            # 초당 최대 요청 수 지정  # 초당 2500개의 요청을 보낼 수 있음
--max_tokens_per_minute 100000 \                            # 초당 사용할 수 있는 최대 토큰 수 지정
--token_encoding_name cl100k_base \                         # OpenAI API에서 사용하는 토큰 인코딩 방식
--max_attempts 5 \                                          # 각 요청에 대해 최대 재시도 횟수
--logging_level 20                                          # 로그 레벨 지정  # 10: DEBUG, 20: INFO, 30: WARNING

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

In [ ]:
# GPT의 평가 결과 JSONL을 불러와서 CSV로 변환 후 정확히 답변한 SQL 쿼리의 갯수 출력
base_eval = change_jsonl_to_csv(f"results/{eval_filepath}", "results/yi_ko_6b_eval.csv", "prompt", "resolve_yn")
base_eval['resolve_yn'] = base_eval['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
num_correct_answers = base_eval.query("resolve_yn == 'yes'").shape[0]
num_correct_answers  # 미세 조정 전 모델 평가 결과: 총 112개 프롬프트 중 정확한 답변 17개 생성

17

In [ ]:
# 학습 데이터 불러와서 csv로 저장
from datasets import load_dataset

df_sql = load_dataset("shangrilar/ko_text2sql", "origin")["train"]  # train 스플릿 데이터 불러옴
df_sql = df_sql.to_pandas()  # 판다스 데이터프레임 형식으로 변환
df_sql = df_sql.dropna().sample(frac=1, random_state=42)  # 결측치 제거하고, frac=1 모든 데이터를 100% 섞음, 랜덤시드 고정
df_sql = df_sql.query("db_id != 1")  # query()는 필터링하는 함수, 가독성은 간결하나 .loc보다 느림(큰 데이터에는 부적절)  # id가 1(게임 카테고리)인 걸 제외하고 가져옴(-> 평가 데이터로 사용)

for idx, row in df_sql.iterrows():  # df_sql 데이터를 순회하면서 인덱스와 각 행을 가져옴
    df_sql.loc[idx, 'text'] = make_prompt(row['context'], row['question'], row['answer'])  # 프롬프트 생성 함수에 각 DDL, 질문, 정답 쿼리를 입력으로 넣고 생성된 프롬프트를 'text'에 저장

!mkdir data  # 새 폴더 생성
df_sql.to_csv('data/train.csv', index=False)  # 제작한 데이터셋을 csv로 저장

In [ ]:
# 미세 조정 명령어
base_model = 'beomi/Yi-Ko-6B'  # 베이스 모델
finetuned_model = 'yi-ko-6b-text2sql'  # 파인튜닝이 완료된 새 모델

!autotrain llm \                       # AutoTrain CLI 명령어
--train \                              # 모델을 학습 및 미세 조정 수행
--model {base_model} \                 # 모델 베이스 경로
--project-name {finetuned_model} \     # 모델 이름(위에서 변수 생성)
--data-path data/ \                    # 학습 데이터가 저장된 경로(위에서 생성한 data 폴더)
--text-column text \                   # 학습 데이터의 text 컬럼(생성한 프롬프트)
--lr 2e-4 \                            # 학습률(Learning Rate), 작은 수치값
--batch-size 8 \                       # 배치 사이즈 설정
--epochs 1 \                           # 에포크 설정
--block-size 1024 \                    # 입력 시퀀스의 최대 길이(토큰 단위)
--warmup-ratio 0.1 \                   # 학습 초기에 학습률을 점진적으로 증가시키는 비율  # 0.1: 전체 학습 중 10% 동안 만 학습률을 서서히 증가 시킴
--lora-r 16 \                          # 로라 설정  # 사용할 랭크, 적응 가능한 파라미터의 크기를 결정
--lora-alpha 32 \                      # 로라 설정  # 로라의 스케일링 팩터
--lora-dropout 0.05 \                  # 로라 설정  # 로라 드롭아웃 확률, 과적합 방지
--weight-decay 0.01 \                  # 가중치 감쇠 비율, 과적합을 방지하지만 수치가 너무 크면 모델 성능에 영향을 줌
--gradient-accumulation 8 \            # 기울기 누적 단계 설정  # 8: 8번의 배치 업데이트 후 기울기 업데이트
--mixed-precision fp16 \               # 혼합 정밀도 학습(Mixed Precision)을 활성화  # 16비트 부동소수점(half-precision)으로 계산하여 GPU 메모리 사용량과 학습 속도를 개선
--use-peft \                           # PEFT(경량화된 미세 조정) 사용
--quantization int4 \                  # 모델 양자화 진행, 4비트 정수 기반으로 모델을 양자화
--trainer sft                          # 학습 방식 선택  # sft: 지도 학습 기반 미세 조정

INFO     | 2025-01-23 15:59:28 | autotrain.cli.run_llm:run:343 - Running LLM
WARNING  | 2025-01-23 15:59:28 | autotrain.trainers.common:__init__:180 - Parameters supplied but not used: func, train, version, inference, config, deploy, backend
Saving the dataset (1/1 shards): 100% 33876/33876 [00:00<00:00, 527797.46 examples/s]
Saving the dataset (1/1 shards): 100% 33876/33876 [00:00<00:00, 537364.81 examples/s]
INFO     | 2025-01-23 15:59:30 | autotrain.backend:create:300 - Starting local training...
INFO     | 2025-01-23 15:59:30 | autotrain.commands:launch_command:327 - ['accelerate', 'launch', '--num_machines', '1', '--num_processes', '1', '--mixed_precision', 'fp16', '-m', 'autotrain.trainers.clm', '--training_config', 'yi-ko-6b-text2sql/training_params.json']
INFO     | 2025-01-23 15:59:30 | autotrain.commands:launch_command:328 - {'model': 'beomi/Yi-Ko-6B', 'project_name': 'yi-ko-6b-text2sql', 'data_path': 'yi-ko-6b-text2sql/autotrain-data', 'train_split': 'train', 'valid_split': 

In [ ]:
# 허깅페이스 로그인
from huggingface_hub import login

# 허깅페이스 허브 로그인
token = "my_token"  # 허깅페이스 액세스 토큰 입력
login(token=token)

In [ ]:
# LoRA 어댑터 결합 및 허깅페이스 허브 업로드
import torch
from peft import LoraConfig, PeftModel

model_name = base_model  # 위에서 지정한 기본 모델
device_map = {"": 0}  # 모델을 로드할 디바이스 설정  # 0: 모델의 모든 파라미터를 GPU 0번 디바이스에 로드

# LoRA와 기초 모델 파라미터 합치기
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,      # CPU 메모리 사용을 최소화하여 모델을 로드
    return_dict=True,            # 모델 출력을 dictionary 형식으로 반환
    torch_dtype=torch.float16,   # 16비트 부동소수점 형식으로 로드(메모리 사용량 감소)
    device_map=device_map,       # GPU 0번 디바이스에 모델을 로드
)
model = PeftModel.from_pretrained(base_model, finetuned_model)  # PEFT를 통해 미세 조정된 LoRA 어댑터를 로드하여 기본 모델(base_model)에 결합
model = model.merge_and_unload()   # LoRA 어댑터 파라미터를 기본 모델에 병합, 병합 후 로라 어댑터는 메모리에서 언(un)로드

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # eos 토큰으로 패딩 토큰 설정
tokenizer.padding_side = "right"  # 력 문장 패딩을 오른쪽에 추가

# 허깅페이스 허브에 모델 및 토크나이저 저장
model.push_to_hub(finetuned_model, use_temp_dir=False)
tokenizer.push_to_hub(finetuned_model, use_temp_dir=False)  # 모델 및 토크나이저 허깅페이스 허브에 업로드  # 임시 디렉토리 지정안하고 현재 경로에서 업로드

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/edgeun/yi-ko-6b-text2sql/commit/28033e0cd79112c763cb8a031703a60f9de2e059', commit_message='Upload tokenizer', commit_description='', oid='28033e0cd79112c763cb8a031703a60f9de2e059', pr_url=None, repo_url=RepoUrl('https://huggingface.co/edgeun/yi-ko-6b-text2sql', endpoint='https://huggingface.co', repo_type='model', repo_id='edgeun/yi-ko-6b-text2sql'), pr_revision=None, pr_num=None)

In [ ]:
# 미세 조정한 모델로 예시 데이터에 대한 SQL 생성
model_id = "edgeun/yi-ko-6b-text2sql"  # 허브에 업로드한 미세 조정 모델 경로
hf_pipe = make_inference_pipeline(model_id)

hf_pipe(example, do_sample=False,
       return_full_text=False, max_length=1024, truncation=True)  # 기초 모델 평가에 사용된 example를 파이프라인에 넣음

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n            "}]

In [ ]:
# 미세 조정한 모델 성능 측정
# sql 생성 수행 (기초 모델 평가와 동일)
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,  # 평가용 test 데이터
                   return_full_text=False, max_length=1024, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

# 미세 조정 모델 평가를 위한 requests.jsonl 생성
ft_eval_filepath = "text2sql_evaluation_finetuned.jsonl"  # 미세 조정 모델용 요청 jsonl 경로 설정
make_requests_for_gpt_evaluation(df, ft_eval_filepath)

# GPT-4 평가 수행(기초 모델 평가와 동일)
!python api_request_parallel_processor.py \
  --requests_filepath requests/{ft_eval_filepath} \  # 미세 조정 모델용 경로
  --save_filepath results/{ft_eval_filepath} \       # 미세 조정 모델용 경로
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 2500 \
  --max_tokens_per_minute 100000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

In [ ]:
ft_eval = change_jsonl_to_csv(f"results/{ft_eval_filepath}", "results/yi_ko_6b_eval.csv", "prompt", "resolve_yn")
ft_eval['resolve_yn'] = ft_eval['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
num_correct_answers = ft_eval.query("resolve_yn == 'yes'").shape[0]
num_correct_answers  # test 데이터 총 112개 중 111개의 쿼리를 해결  # 미세 조정 전 15.17% -> 미세 조정 후 모델 테스트 99.1%

111